Como o arquivo é baixado é muito grande para subir no Git apagamos e deixamos para que o usuario possa fazer o download.

In [1]:
from download_dados import ColigacaoDownloader 

cd = ColigacaoDownloader()
cd.download(['2022'])
cd.unzip(['2022'], 'SP')

Descompactando zipfile: 100%|██████████| 1/1 [00:00<00:00, 56.81it/s]


In [2]:
from download_dados import CandidatoDownloader 

cd = CandidatoDownloader()
cd.download(['2022'])
cd.unzip(['2022'], 'SP')

Descompactando zipfile: 100%|██████████| 1/1 [00:00<00:00, 60.90it/s]


In [3]:
from download_dados import VotacaoSecaoDownloader 

cd = VotacaoSecaoDownloader()
cd.download(['2022'], 'SP')
cd.unzip(['2022'], 'SP')

In [ ]:
from download_dados import ColigacaoDownloader 

cd = ColigacaoDownloader()
cd.download(['2022'])
cd.unzip(['2022'], 'SP')

In [ ]:
from download_dados import PrestacaoDeContasDownloader 

cd = PrestacaoDeContasDownloader()
cd.download(['2022'])
cd.unzip(['2022'], 'SP')

In [ ]:
from download_dados import PrestacaoDeContasDownloader

cd = PrestacaoDeContasDownloader()
cd.unzip_receita_candidatos(['2022'], 'SP')

In [ ]:
import os

def save_csv(arq_nome, cargo, uf, ano, df, turno=None):
    if turno:
        path = os.path.join('dados', 'processados', uf, ano, cargo, turno)
    else:
        path = os.path.join('dados', 'processados', uf, ano, cargo)

    if not os.path.exists(path):
        os.makedirs(path)
    
    path = os.path.join(path, f'{arq_nome}.csv')
    df.to_csv(path, encoding='latin-1', index=False)

In [ ]:
import pandas as pd

df_coligacao = pd.read_csv('dados/brutos/SP/2022/consulta_coligacao_2022_SP.csv', encoding='latin-1', sep=';')
df_coligacao.drop(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO', 'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'SG_UF', 'SG_UE', 'NM_UE'], axis=1, inplace=True)

df_coligacao_df = df_coligacao[(df_coligacao['CD_CARGO']==6)]
save_csv('coligacao', 'DeputadoFederal', 'SP', '2022', df_coligacao_df)

df_coligacao_de = df_coligacao[(df_coligacao['CD_CARGO']==7)]
save_csv('coligacao', 'DeputadoEstadual', 'SP', '2022', df_coligacao_de)

In [ ]:
import pandas as pd

df_cand = pd.read_csv('dados/brutos/SP/2022/consulta_cand_2022_SP.csv', encoding='latin-1', sep=';')
df_cand_df = df_cand[(df_cand['CD_CARGO']==6)]
df_cand_df = df_cand.drop(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO', 'NM_TIPO_ELEICAO', 
                           'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'TP_ABRANGENCIA', 'SG_UF', 
                           'SG_UE', 'NM_UE', 'DS_CARGO', 'NM_URNA_CANDIDATO', 'NM_SOCIAL_CANDIDATO', 'NR_CPF_CANDIDATO', 
                           'NM_EMAIL', 'NR_CANDIDATO', 'NM_MUNICIPIO_NASCIMENTO', 'CD_NACIONALIDADE', 'DS_NACIONALIDADE', 
                           'SG_UF_NASCIMENTO', 'CD_MUNICIPIO_NASCIMENTO', 'NR_TITULO_ELEITORAL_CANDIDATO', 'DT_NASCIMENTO',], axis=1)
save_csv('candidato', 'DeputadoFederal', 'SP', '2022',df_cand_df)

In [ ]:
import pandas as pd

df = pd.read_csv('dados/brutos/SP/2022/receitas_candidatos_2022_SP.csv', encoding='latin-1', sep=';')

df_receita_cand = df[(df['CD_CARGO']==6)]
df_receita_cand['VR_RECEITA'] = df_receita_cand['VR_RECEITA'].str.replace(',', '.').astype(float)
df_receita_cand = df_receita_cand.groupby(['SQ_CANDIDATO', 'CD_FONTE_RECEITA', 'DS_FONTE_RECEITA', 'CD_ORIGEM_RECEITA', 'DS_ORIGEM_RECEITA']).agg({'VR_RECEITA':'sum'}).reset_index()
save_csv('receita_candidato', 'DeputadoFederal', 'SP', '2022', df_receita_cand)

df_doadores_df = df[(df['CD_CARGO']==6) & (df['CD_ORIGEM_RECEITA']==10010200)]
df_doadores_df['VR_RECEITA'] = df_doadores_df['VR_RECEITA'].str.replace(',', '.').astype(float)
df_doadores_df = df_doadores_df.groupby(['NR_CPF_CNPJ_DOADOR', 'NM_DOADOR', 'NM_DOADOR_RFB', 'DS_RECEITA']).agg({'VR_RECEITA':'sum'}).reset_index()
save_csv('doadores', 'DeputadoFederal', 'SP', '2022', df_doadores_df)

df_doacao_cand = df[(df['CD_CARGO']==6) & (df['CD_ORIGEM_RECEITA']==10010200)]
df_doacao_cand['VR_RECEITA'] = df_doacao_cand['VR_RECEITA'].str.replace(',', '.').astype(float)
df_doacao_cand.groupby(['SQ_CANDIDATO', 'NR_CPF_CNPJ_DOADOR', 'NM_DOADOR', 'NM_DOADOR_RFB', 'DS_RECEITA']).agg({'VR_RECEITA':'sum'}).reset_index()
save_csv('doador_candidato', 'DeputadoFederal', 'SP', '2022', df_doacao_cand)

In [ ]:
df_doadores_df = df[(df['CD_CARGO']==6) & (df['CD_ORIGEM_RECEITA']==10010200)]
df_doadores_df['VR_RECEITA'] = df_doadores_df['VR_RECEITA'].str.replace(',', '.').astype(float)
df_doadores_df = df_doadores_df.groupby(['SQ_CANDIDATO', 'CD_FONTE_RECEITA', 'DS_FONTE_RECEITA', 'CD_ORIGEM_RECEITA', 'DS_ORIGEM_RECEITA']).agg({'VR_RECEITA':'sum'}).reset_index()
#save_csv('doadores', 'DeputadoFederal', 'SP', '2022', df_doacao_cand_df)

In [ ]:
import pandas as pd

df_votos_secao = pd.read_csv('dados/brutos/SP/2022/votacao_secao_2022_SP.csv', encoding='latin-1', sep=';')

#df_votos_partido_de = df_votos_secao[(df_votos_secao['SQ_CANDIDATO']==-3) & (df_votos_secao['CD_CARGO']==7)]
#df_votos_partido_de = df_votos_partido_de.groupby(['SQ_CANDIDATO', 'NM_VOTAVEL', 'NR_VOTAVEL']).agg({'QT_VOTOS':'sum'}).reset_index()
#save_csv('votacao_partido', 'DeputadoEstadual', 'SP', '2022', df_votos_partido_de)

df_votos_partido_df = df_votos_secao[(df_votos_secao['SQ_CANDIDATO']==-3) & (df_votos_secao['CD_CARGO']==6)]
df_votos_partido_df = df_votos_partido_df.groupby(['SQ_CANDIDATO', 'NM_VOTAVEL', 'NR_VOTAVEL']).agg({'QT_VOTOS':'sum'}).reset_index()
save_csv('votacao_partido', 'DeputadoFederal', 'SP', '2022', df_votos_partido_df)

#df_votos_candidato_df = df_votos_secao[(~df_votos_secao['SQ_CANDIDATO'].isin([-3,-1])) & (df_votos_secao['CD_CARGO']==6)]
#df_votos_candidato_df = df_votos_candidato_df.groupby(['SQ_CANDIDATO',  'NM_VOTAVEL', 'NR_VOTAVEL']).agg({'QT_VOTOS':'sum'}).reset_index()
#save_csv('votacao_candidato', 'DeputadoFederal', 'SP', '2022', df_votos_candidato_df)

#df_votos_candidato_de = df_votos_secao[(~df_votos_secao['SQ_CANDIDATO'].isin([-3,-1])) & (df_votos_secao['CD_CARGO']==7)]
#df_votos_candidato_de = df_votos_candidato_de.groupby(['SQ_CANDIDATO', 'NM_VOTAVEL', 'NR_VOTAVEL']).agg({'QT_VOTOS':'sum'}).reset_index()
#save_csv('votacao_candidato', 'DeputadoEstadual', 'SP', '2022', df_votos_candidato_de)

#df_votos_candidato_gv1 = df_votos_secao[(df_votos_secao['CD_CARGO']==3) & (df_votos_secao['NR_TURNO']==1)]
#df_votos_candidato_gv1 = df_votos_candidato_gv1.groupby(['SQ_CANDIDATO', 'NM_VOTAVEL', 'NR_VOTAVEL']).agg({'QT_VOTOS':'sum'}).reset_index()
#save_csv('votacao_candidato', 'Governador', 'SP', '2022', df_votos_candidato_gv1, '1_TURNO')

#df_votos_candidato_gv2 = df_votos_secao[(df_votos_secao['CD_CARGO']==3) & (df_votos_secao['NR_TURNO']==2)]
#df_votos_candidato_gv2 = df_votos_candidato_gv2.groupby(['SQ_CANDIDATO', 'NM_VOTAVEL', 'NR_VOTAVEL']).agg({'QT_VOTOS':'sum'}).reset_index()
#save_csv('votacao_candidato', 'Governador', 'SP', '2022', df_votos_candidato_gv1, '2_TURNO')

<center>
    <img src="https://nucleodetecnologia.com.br/assets/img/novo-logo-tecnologia.svg?bl=tutorial-eleitoral-01" width="150">
</center>